In [342]:
import pandas as pd
import numpy as np


In [343]:
### data inladen
wedstrijden = pd.read_excel('../../Data/Silver/wedstrijden_cleaned.xlsx')
corners = pd.read_excel('../../Data/Silver/cornerballen_cleaned.xlsx')

merged_df = pd.merge(wedstrijden, corners, on='wedstrijd_id', how='inner')


In [344]:
# voor merged_df gaan we alle kolommen met een v in opsplitsen in thuis_kolom en uit_kolom
merged_df[['klassement_thuis', 'klassement_uit']] = merged_df['klassement_uitslag'].str.split('v', expand=True)
merged_df[['uitslag_thuis', 'uitslag_uit']] = merged_df['uitslag'].str.split('v', expand=True)
merged_df[['score_voor_thuis', 'score_voor_uit']] = merged_df['score_voor'].str.split('v', expand=True)

# Convert to numeric
cols_to_convert = [
    'klassement_thuis', 'klassement_uit',
    'uitslag_thuis', 'uitslag_uit',
    'score_voor_thuis', 'score_voor_uit'
]

for col in cols_to_convert:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').astype('Int64')

# drop de kolommen die we niet meer nodig hebben
merged_df = merged_df.drop(columns=['klassement_uitslag', 'uitslag', 'score_voor'])


In [345]:
merged_df.dtypes

wedstrijd_id                       int64
datum                     datetime64[ns]
niveau                            object
corners_uitslag                   object
schepcorner_id                     int64
ploegnaam                         object
ervaring_schepper                float64
ervaring_kopper                  float64
ervaring_duo                     float64
is_thuisploeg                      int64
kwart                              int64
is_schepper_bank                   int64
is_kopper_bank                     int64
schepper_pos_rechts                int64
positie                            int64
kop_verplaatsen_achter             int64
kop_veel_verplaatsen               int64
kop_hor_verplaatsen                int64
goal                               int64
klassement_thuis                   Int64
klassement_uit                     Int64
uitslag_thuis                      Int64
uitslag_uit                        Int64
score_voor_thuis                   Int64
score_voor_uit  

In [346]:
wedstrijden.head()

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag
0,1,2025-03-15,4,4v1,2v1,1v5
1,2,2024-09-03,4,11v2,2v1,8v3
2,3,2024-09-03,4,4v6,1v1,3v10
3,4,2024-09-03,3,5v3,1v3,4v9
4,5,2024-09-03,4,7v2,0v2,NaN


In [347]:
corners.head()

,schepcorner_id,wedstrijd_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
0,1,1,marathon,4.0,4.0,4.0,1,1,1,0,0v0,1,5,0,0,0,1
1,2,1,marathon,4.0,4.0,4.0,1,2,0,0,2v0,1,5,0,0,0,0
2,3,1,poba juniors,3.0,2.0,3.0,0,3,0,0,2v0,0,2,0,0,1,0
3,4,2,mvc rudie,4.0,5.0,4.0,1,3,1,0,4v1,1,5,0,1,1,0
4,5,2,uncle abes patty pounders,1.0,5.0,1.0,0,3,1,0,5v2,1,5,0,0,0,0


### data exploration

In [348]:
### missing values in de merged_df
print(merged_df.isnull().sum()/len(merged_df)*100)

wedstrijd_id               0.000000
datum                      0.000000
niveau                     0.000000
corners_uitslag           34.328358
schepcorner_id             0.000000
ploegnaam                  0.000000
ervaring_schepper         29.519071
ervaring_kopper           29.850746
ervaring_duo              29.519071
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
klassement_thuis          41.956882
klassement_uit            41.956882
uitslag_thuis              0.000000
uitslag_uit                0.000000
score_voor_thuis           0.000000
score_voor_uit             0.000000
dtype: float64


In [349]:
print(wedstrijden.isna().sum()/len(merged_df)*100)

wedstrijd_id           0.000000
datum                  0.000000
niveau                 0.000000
uitslag                0.000000
corners_uitslag       12.935323
klassement_uitslag    16.749585
dtype: float64


In [350]:
print(corners.isna().sum()/len(merged_df)*100)

schepcorner_id             0.000000
wedstrijd_id               0.000000
ploegnaam                  0.000000
ervaring_schepper         29.519071
ervaring_kopper           29.850746
ervaring_duo              29.519071
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
score_voor                 0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
dtype: float64


In [351]:
### veel missing values bij ervaring, hoe imputen?
# 1 imputen met vorige wedstrijden, anders met gemiddelde van niveau
# 2 prediction model bouwen

### missing values in corners_uistlag, geen probleem wordt niet gebruikt in de analyse
# kolom droppen

### klassement uitslag missing values, hoe imputen?
# 1 imputen met vorige wedstrijden, of gemiddelde afstand gebruiken
# 2 anders met finale uitslag van klassement, nadeel veel manueel werk

In [352]:
# droppen corners_uitslag kolom
merged_df.drop(columns=['corners_uitslag'], inplace=True)


In [353]:
merged_df.head()

,wedstrijd_id,datum,niveau,schepcorner_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,...,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal,klassement_thuis,klassement_uit,uitslag_thuis,uitslag_uit,score_voor_thuis,score_voor_uit
0,1,2025-03-15,4,1,marathon,4.0,4.0,4.0,1,1,...,0,0,0,1,1,5,4,1,0,0
1,1,2025-03-15,4,2,marathon,4.0,4.0,4.0,1,2,...,0,0,0,0,1,5,4,1,2,0
2,1,2025-03-15,4,3,poba juniors,3.0,2.0,3.0,0,3,...,0,0,1,0,1,5,4,1,2,0
3,2,2024-09-03,4,4,mvc rudie,4.0,5.0,4.0,1,3,...,0,1,1,0,8,3,11,2,4,1
4,2,2024-09-03,4,5,uncle abes patty pounders,1.0,5.0,1.0,0,3,...,0,0,0,0,8,3,11,2,5,2


In [354]:
# Imputatiefunctie
def imputatie_rij(rij, ervaring_col, ploegnaam_means, niveau_means, mean_overall):
    if pd.notnull(rij[ervaring_col]):
        return rij[ervaring_col]

    ploegnaam_mean = ploegnaam_means.get(rij['ploegnaam'], np.nan)
    if pd.notnull(ploegnaam_mean):
        return ploegnaam_mean

    niveau_mean = niveau_means.get(rij['niveau'], np.nan)
    if pd.notnull(niveau_mean):
        return niveau_mean

    return mean_overall

In [355]:
# imputen van ervaring
def impute_ervaring(df, ervaring_col):
    """
    imputen met gemiddelde vorige wedstrijden van ploeg, anders met gemiddelde van niveau
    """

    df = df.copy()

    ploegnaam_means = df.groupby('ploegnaam')[ervaring_col].mean()
    niveau_means = df.groupby('niveau')[ervaring_col].mean()
    mean_overall = df[ervaring_col].mean()

    df[ervaring_col] = df.apply(
        imputatie_rij,
        axis=1,
        ervaring_col=ervaring_col,
        ploegnaam_means=ploegnaam_means,
        niveau_means=niveau_means,
        mean_overall=mean_overall
    )

    return df

In [356]:
ervaring_imputed_df = impute_ervaring(merged_df, 'ervaring_schepper')
ervaring_imputed_df = impute_ervaring(ervaring_imputed_df, 'ervaring_kopper')
ervaring_imputed_df = impute_ervaring(ervaring_imputed_df, 'ervaring_duo')


In [357]:
print(ervaring_imputed_df.isnull().sum()/len(merged_df)*100)

wedstrijd_id               0.000000
datum                      0.000000
niveau                     0.000000
schepcorner_id             0.000000
ploegnaam                  0.000000
ervaring_schepper          0.000000
ervaring_kopper            0.000000
ervaring_duo               0.000000
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
klassement_thuis          41.956882
klassement_uit            41.956882
uitslag_thuis              0.000000
uitslag_uit                0.000000
score_voor_thuis           0.000000
score_voor_uit             0.000000
dtype: float64


In [358]:
ervaring_imputed_df[ervaring_imputed_df['ervaring_schepper'].isnull()]

,wedstrijd_id,datum,niveau,schepcorner_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,...,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal,klassement_thuis,klassement_uit,uitslag_thuis,uitslag_uit,score_voor_thuis,score_voor_uit


In [359]:
# impute p2 met gemiddelde overal

In [360]:
### TODO: klassement imputeren
# eerst klassement_uitslag opsplitsen in thuis_klassement en uit_kassement
ervaring_imputed_df[["ploegnaam", "datum",'klassement_thuis', 'klassement_uit', "is_thuisploeg"]]
ervaring_imputed_df["klassement"] = np.where(ervaring_imputed_df["is_thuisploeg"] == 1, ervaring_imputed_df["klassement_thuis"], ervaring_imputed_df["klassement_uit"])


In [361]:
ervaring_imputed_df[["ploegnaam", "datum",'klassement_thuis', 'klassement_uit', "is_thuisploeg", "klassement"]]

,ploegnaam,datum,klassement_thuis,klassement_uit,is_thuisploeg,klassement
0,marathon,2025-03-15,1,5,1,1.0
1,marathon,2025-03-15,1,5,1,1.0
2,poba juniors,2025-03-15,1,5,0,5.0
3,mvc rudie,2024-09-03,8,3,1,8.0
4,uncle abes patty pounders,2024-09-03,8,3,0,3.0
...,...,...,...,...,...,...
598,sint gillis waas,2024-11-29,10,12,0,12.0
599,erwetegem,2024-12-06,5,2,0,2.0
600,erwetegem,2024-12-06,5,2,0,2.0
601,erwetegem,2024-12-06,5,2,0,2.0


In [362]:
ervaring_imputed_df["datum"]= pd.to_datetime(ervaring_imputed_df["datum"])

# Rijen met en zonder NA
na_rows = ervaring_imputed_df[ervaring_imputed_df['klassement'].isna()].copy()
notna_rows = ervaring_imputed_df[ervaring_imputed_df['klassement'].notna()].copy()


In [363]:

# Functie om dichtstbijzijnde klassement te zoeken
def impute_klassement(row):
    ploeg = row['ploegnaam']
    datum = row['datum']
    
    kandidaten = notna_rows[notna_rows['ploegnaam'] == ploeg]
    if kandidaten.empty:
        return np.nan  # geen enkele match gevonden
    
    # Bereken absolute tijdsverschil
    kandidaten['datumverschil'] = (kandidaten['datum'] - datum).abs()
    
    # Neem de rij met het kleinste tijdsverschil
    beste_match = kandidaten.loc[kandidaten['datumverschil'].idxmin()]
    
    return beste_match['klassement']


In [364]:

# Imputeren
na_rows['klassement'] = na_rows.apply(impute_klassement, axis=1)

# Combineer terug
df_imputed = pd.concat([notna_rows, na_rows]).sort_index()

In [365]:
df_imputed.head()

,wedstrijd_id,datum,niveau,schepcorner_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,...,kop_veel_verplaatsen,kop_hor_verplaatsen,goal,klassement_thuis,klassement_uit,uitslag_thuis,uitslag_uit,score_voor_thuis,score_voor_uit,klassement
0,1,2025-03-15,4,1,marathon,4.0,4.0,4.0,1,1,...,0,0,1,1,5,4,1,0,0,1.0
1,1,2025-03-15,4,2,marathon,4.0,4.0,4.0,1,2,...,0,0,0,1,5,4,1,2,0,1.0
2,1,2025-03-15,4,3,poba juniors,3.0,2.0,3.0,0,3,...,0,1,0,1,5,4,1,2,0,5.0
3,2,2024-09-03,4,4,mvc rudie,4.0,5.0,4.0,1,3,...,1,1,0,8,3,11,2,4,1,8.0
4,2,2024-09-03,4,5,uncle abes patty pounders,1.0,5.0,1.0,0,3,...,0,0,0,8,3,11,2,5,2,3.0


In [366]:
df_imputed.isnull().sum()/len(df_imputed)*100

wedstrijd_id               0.000000
datum                      0.000000
niveau                     0.000000
schepcorner_id             0.000000
ploegnaam                  0.000000
ervaring_schepper          0.000000
ervaring_kopper            0.000000
ervaring_duo               0.000000
is_thuisploeg              0.000000
kwart                      0.000000
is_schepper_bank           0.000000
is_kopper_bank             0.000000
schepper_pos_rechts        0.000000
positie                    0.000000
kop_verplaatsen_achter     0.000000
kop_veel_verplaatsen       0.000000
kop_hor_verplaatsen        0.000000
goal                       0.000000
klassement_thuis          41.956882
klassement_uit            41.956882
uitslag_thuis              0.000000
uitslag_uit                0.000000
score_voor_thuis           0.000000
score_voor_uit             0.000000
klassement                27.694859
dtype: float64

In [367]:
### wat te doen nog steeds 28% missing values?

In [368]:
ervaring_imputed_df.shape

(603, 25)

### Feature Engineering

In [369]:
basetable = ervaring_imputed_df.copy()

In [370]:
def is_degradatieplaats(rij):
    niveau = str(rij['niveau']).lower()
    klassement = rij['klassement']
    
    # Niveau 4: geen degradatie
    if niveau == '4':
        return 0
    # Niveau 1 of 2: degradatie vanaf plaats 13
    elif niveau in ['1', '2']:
        return int(klassement >= 13)
    # Niveau 3: degradatie vanaf plaats 14
    elif niveau == '3':
        return int(klassement >= 14)
    # Niveau n1: degradatie bij plaats 11 of 12
    elif niveau == 'n1':
        return int(klassement in [11, 12])
    # Alle andere niveaus: degradatie vanaf plaats 14
    else:
        return int(klassement >= 14)


In [371]:
def is_promotieplaats(rij):
    klassement = rij['klassement']
    return int(klassement in [1, 2, 3])


In [372]:
# klassement positie 
basetable["klassement_veschil"] = abs(basetable["klassement_thuis"] - basetable["klassement_uit"])
basetable["is_degradatieplaats"] = basetable.apply(is_degradatieplaats, axis=1)
basetable["is_promotieplaats"] = basetable.apply(is_promotieplaats, axis=1)



In [373]:
#score_verschil_voor
basetable["score_verschil_voor"] = abs(basetable["score_voor_thuis"] - basetable["score_voor_uit"])

In [374]:
# periode van het seizoen TODO: is dit wel relevant? veel data in het begin van het seizoen mss beter gebaseerd op maanden
# Functie om periode toe te kennen op basis van maand
def maand_periode(datum):
    maand = datum.month
    if maand in [8, 9]:      # augustus, september
        return 1
    elif maand in [10, 11]:  # oktober, november
        return 2
    elif maand in [12, 1]:   # december, januari
        return 3
    elif maand in [2, 3, 4]: # februari, maart, april
        return 4
    else:
        return None  # mei–juli vallen buiten seizoen

# Toepassen op de basetable
basetable["periode"] = basetable["datum"].apply(maand_periode)

In [375]:
# de hoeveelste schepcorner van de wedstrijd
basetable["schepcorner_nummer"] = basetable.groupby("wedstrijd_id").cumcount() + 1

In [376]:
basetable["schepcorner_nummer"].unique()

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [377]:
basetable.columns

Index(['wedstrijd_id', 'datum', 'niveau', 'schepcorner_id', 'ploegnaam',
       'ervaring_schepper', 'ervaring_kopper', 'ervaring_duo', 'is_thuisploeg',
       'kwart', 'is_schepper_bank', 'is_kopper_bank', 'schepper_pos_rechts',
       'positie', 'kop_verplaatsen_achter', 'kop_veel_verplaatsen',
       'kop_hor_verplaatsen', 'goal', 'klassement_thuis', 'klassement_uit',
       'uitslag_thuis', 'uitslag_uit', 'score_voor_thuis', 'score_voor_uit',
       'klassement', 'klassement_veschil', 'is_degradatieplaats',
       'is_promotieplaats', 'score_verschil_voor', 'periode',
       'schepcorner_nummer'],
      dtype='object')

In [378]:
# lag toevoegen van de vorige schepcorner
lag_df = basetable[["wedstrijd_id", "schepcorner_id", "datum", "ploegnaam", "goal"]].copy()

lag_df = lag_df.sort_values(["datum", "schepcorner_id"]).reset_index()
lag_df["goal_lag_ploeg"] = lag_df.groupby("ploegnaam")["goal"].shift(1)
lag_df["goal_lag_wedstrijd"] = lag_df.groupby("wedstrijd_id")["goal"].shift(1)
gemiddeld_scorepercentage = lag_df["goal"].mean()

lag_df["goal_lag_ploeg"] = lag_df["goal_lag_ploeg"].fillna(gemiddeld_scorepercentage)
lag_df["goal_lag_wedstrijd"] = lag_df["goal_lag_wedstrijd"].fillna(gemiddeld_scorepercentage)

# merge met de basetable
basetable = basetable.merge(lag_df[["schepcorner_id", "goal_lag_ploeg", "goal_lag_wedstrijd"]], on="schepcorner_id", how="left")

basetable[basetable["ploegnaam"] == "fc spitbulls"][["wedstrijd_id", "schepcorner_id", "datum", "ploegnaam", "goal", "goal_lag_ploeg", "goal_lag_wedstrijd"]].head(20)

,wedstrijd_id,schepcorner_id,datum,ploegnaam,goal,goal_lag_ploeg,goal_lag_wedstrijd
7,3,8,2024-09-03,fc spitbulls,1,0.000000,0.000000
9,3,10,2024-09-03,fc spitbulls,1,1.000000,1.000000
75,32,76,2025-04-02,fc spitbulls,1,0.000000,0.661692
77,32,78,2025-04-02,fc spitbulls,1,1.000000,1.000000
78,32,79,2025-04-02,fc spitbulls,1,1.000000,1.000000
79,33,80,2025-03-27,fc spitbulls,1,1.000000,0.661692
81,33,82,2025-03-27,fc spitbulls,0,1.000000,0.000000
82,34,83,2025-03-21,fc spitbulls,1,1.000000,0.661692
85,35,86,2025-03-11,fc spitbulls,1,1.000000,1.000000
87,36,88,2025-03-06,fc spitbulls,1,0.000000,1.000000


In [379]:
# aggregate niveaus, national, kern_hoog, kern_laag
basetable["niveau"] = basetable["niveau"].replace({"n1": "nationaal", "n4": "nationaal","p2": "nationaal", 1: "kern_hoog", 2: "kern_hoog", 3: "kern_laag", 4: "kern_laag"})

In [380]:
basetable.columns

Index(['wedstrijd_id', 'datum', 'niveau', 'schepcorner_id', 'ploegnaam',
       'ervaring_schepper', 'ervaring_kopper', 'ervaring_duo', 'is_thuisploeg',
       'kwart', 'is_schepper_bank', 'is_kopper_bank', 'schepper_pos_rechts',
       'positie', 'kop_verplaatsen_achter', 'kop_veel_verplaatsen',
       'kop_hor_verplaatsen', 'goal', 'klassement_thuis', 'klassement_uit',
       'uitslag_thuis', 'uitslag_uit', 'score_voor_thuis', 'score_voor_uit',
       'klassement', 'klassement_veschil', 'is_degradatieplaats',
       'is_promotieplaats', 'score_verschil_voor', 'periode',
       'schepcorner_nummer', 'goal_lag_ploeg', 'goal_lag_wedstrijd'],
      dtype='object')

In [381]:
columns_to_drop = ["wedstrijd_id"]
basetable.drop(columns=columns_to_drop, inplace=True)

In [382]:
# schrijf weg
basetable.to_csv('../../Data/Gold/basetable.csv', index=False)